In [1]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
sys.path.append('../')

sys.path.append('../../REA_with_CESM2')
from ensembles.ensemble_GKLT import ensemble_GKLT
from data_extractor.data_extractor import extract_rea,extract_initial

from experiment_configuration.experiment import experiment

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'extract_rea' from 'data_extractor.data_extractor' (/home/u/u290372/projects/REA_heat_wEU_JJA/data_extraction/../../REA_with_CESM2/data_extractor/data_extractor.py)

In [ ]:
variable = 'U200'
time_frequency = 'mon'
realm = 'atm'
h_identifier = 'h1'
overwrite = False


In [ ]:
preprocessing = None
preprocessing_module = importlib.import_module(f"preprocessing.{preprocessing}")


ModuleNotFoundError: No module named 'preprocessing.None'

In [15]:
preprocessing_module = None

In [11]:
for experiment_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    print(experiment_identifier)
    # load experiment configuration settings
    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)

    l = extract_rea(
        exp,
        variable = variable,
        realm = realm,
        h_identifier = h_identifier,
        time_frequency = time_frequency,
        preprocessing_module = preprocessing_module,
        overwrite = overwrite,
    )

    if '1' in experiment_identifier:
        extract_initial(
            exp,
            variable = variable,
            realm = realm,
            h_identifier = h_identifier,
            time_frequency = time_frequency,
            preprocessing_module = preprocessing_module,
            overwrite = overwrite,
        )
    



c1
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens001/ua200_mon_CESM2_ssp370-2025-x1_ens001_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens002/ua200_mon_CESM2_ssp370-2025-x1_ens002_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens003/ua200_mon_CESM2_ssp370-2025-x1_ens003_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens004/ua200_mon_CESM2_ssp370-2025-x1_ens004_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens005/ua200_mon_CESM2_ssp370-2025-x1_ens005_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens006/ua200_mon_CESM2_ssp370-2025-x1_ens006_2025.nc
/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/ssp370-2025-x1/mon/atmos/ua200/ens007/ua200_mon_CESM2_ssp370-2025-x1_ens007_2025.nc
/work/bb1152/u290

TypeError: extract_initial() got an unexpected keyword argument 'time_frequency'

In [17]:
for experiment_identifier in [f"c{i}" for i in [1]] + [f"p{i}" for i in [1]]:
    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)

    extract_initial(
        exp,
        variable = variable,
        realm = realm,
        h_identifier = h_identifier,
        time_frequency = time_frequency,
        preprocessing_module = preprocessing_module,
        overwrite = overwrite,
    )

In [7]:
l

[]

In [9]:
experiment_identifier

'c1'